In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from importlib import reload
from config import config_imp as conf
import scripts.raster_analysis as ras
from scripts.model import get_settings, get_prediction_settings
import rasterio as rio
import torch

In [ ]:
SETTING_NAME = "imp_test_1"
(
    model,
    model_name,
    optimizer,
    loss_fn,
    scheduler,
    variable,
    batch_size,
    rescale_factor,
    metadata,
) = get_settings(SETTING_NAME)

stride, width, height, batch_size, pred_metadata = get_prediction_settings(SETTING_NAME)
metadata

In [ ]:
model_path = sorted(
    list([(file.stat().st_atime, file) for file in conf.out_model_path.glob("*9911*")])
)[-1][1]
model_path.name

In [ ]:
model.load_state_dict(torch.load(model_path))
model.eval()
model.to("cuda");

In [ ]:
all_files = list(conf.imgs_to_predict.glob(f"planet*.tif"))
len(all_files)

In [ ]:
for image_path in all_files:
    
    output_name = image_path.with_name(f"pred_{image_path.stem}_strd{stride}.tif").name
    output_file = conf.out_prediction/output_name

    with rio.open(image_path) as img:
        
        profile = img.profile
        mask = ras.detect(
            model, img, width=width, height=height, stride=stride, batch_size=batch_size
        )
        ras.save_mask(mask, profile, output_file)